<a href="https://www.kaggle.com/code/kenny3s/enc-dec-times?scriptVersionId=168077403" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
dataset_dir = "./picture_materials"

In [10]:
import os
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import numpy as np
from skimage import img_as_ubyte, io

# 获取dataset_dir目录下的所有文件名
image_files = os.listdir(dataset_dir)
# 转换为灰度图像
for image_file in image_files:
    image_path = os.path.join(dataset_dir, image_file)
    image = Image.open(image_path)
    gray_images = image.convert("L")
    gray_images.save(os.path.join(dataset_dir, image_file))

In [11]:
# def block_no_expand(PIL_img):
#     image=np.array(PIL_img)
#     image_lab=cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
#     image_lab[:,:,-1]=np.interp(image_lab[:,:,-1],(0,255),(0,255/4))
#     image_rgb=cv2.cvtColor(image_lab,cv2.COLOR_HSV2RGB)
#     image_ht=Image.fromarray(image_rgb).convert("1")
#     image_ht=img_as_ubyte(image_ht)
#     return image_ht
def block_no_expand(PIL_img):
    image = np.array(PIL_img)
    # Scale the image intensity
    image = np.interp(image, (0, 255), (0, 255/4))
    # Convert the image to binary
    image_ht = Image.fromarray(image).convert("1")
    # Convert the binary image to 8-bit unsigned integer format
    image_ht = img_as_ubyte(image_ht)
    return image_ht

In [13]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP, DES3, AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
import time

# RSA加密和解密
def rsa_enc_dec(image):
    # 将图像转换为字节流
    start = time.perf_counter()

    # 生成RSA密钥
    key = RSA.generate(2048)
    cipher = PKCS1_OAEP.new(key)

    # 计算最大明文长度
    max_length = 128

    # 将图像分割成小块
    chunks = [image[i:i+max_length] for i in range(0, len(image), max_length)]

    # 加密
    encrypted_chunks = [cipher.encrypt(chunk) for chunk in chunks]
    enc_time = time.perf_counter() - start

    # 解密
    start = time.perf_counter()
    decrypted_chunks = [cipher.decrypt(chunk) for chunk in encrypted_chunks]
    dec_time = time.perf_counter() - start

    # 将解密后的小块重新组合成完整的图像
    decrypted = b''.join(decrypted_chunks)

    return enc_time, dec_time

def des3_enc_dec(image):
    # 将图像转换为字节流
    start = time.perf_counter()


    # 生成3DES密钥和初始化向量
    key = get_random_bytes(24)
    iv = get_random_bytes(8)

    # 创建一个新的DES3对象，使用CBC模式
    cipher_enc = DES3.new(key, DES3.MODE_CBC, iv)

    # 加密
    encrypted = cipher_enc.encrypt(pad(image, DES3.block_size))
    enc_time = time.perf_counter() - start

    # 创建一个新的DES3对象，使用CBC模式，用于解密
    cipher_dec = DES3.new(key, DES3.MODE_CBC, iv)

    # 解密
    start = time.perf_counter()
    decrypted = unpad(cipher_dec.decrypt(encrypted), DES3.block_size)
    dec_time = time.perf_counter() - start

    return enc_time, dec_time

# AES加密和解密
def aes_enc_dec(image):
    # 将图像转换为字节流
    start = time.perf_counter()

    # 生成AES密钥和初始化向量
    key = get_random_bytes(32)
    iv = get_random_bytes(16)

    # 创建一个新的AES对象，使用CBC模式
    cipher_enc = AES.new(key, AES.MODE_CBC, iv)

    # 加密
    start = time.perf_counter()
    encrypted = cipher_enc.encrypt(pad(image, AES.block_size))
    enc_time = time.perf_counter() - start

    # 创建一个新的AES对象，使用CBC模式，用于解密
    cipher_dec = AES.new(key, AES.MODE_CBC, iv)

    # 解密
    start = time.perf_counter()
    decrypted = unpad(cipher_dec.decrypt(encrypted), AES.block_size)
    dec_time = time.perf_counter() - start

    return enc_time, dec_time

def no_expand_enc_dec(img):
    # 加密
    start = time.perf_counter()
    #img = img.convert('RGB')
    encrypted = block_no_expand(img)
    enc_time = time.perf_counter() - start

    enc_size = Image.fromarray(encrypted).save('encrypted.png')
    print(f'Encrypted image size: {os.path.getsize("encrypted.png") / 1024:.2f} KB')
    # 解密
    start = time.perf_counter()
    dec_time = time.perf_counter() - start

    return enc_time, dec_time

# 对每张图像进行加密和解密，并计算耗时
for image_file in image_files:
    ori_size = os.path.getsize(os.path.join(dataset_dir, image_file))
    # 打开图像文件，并转换为灰度图像
    img_gray = Image.open(os.path.join(dataset_dir, image_file)).convert('L')
    img = img_gray.tobytes()

    img_rgb = Image.open(os.path.join(dataset_dir, image_file))
    # 计算RSA加密和解密的耗时
    rsa_enc_time, rsa_dec_time = rsa_enc_dec(img)
    rsa_enc_time, rsa_dec_time = (round(rsa_enc_time * 1000, 2), round(rsa_dec_time * 1000, 2))  # 转换为毫秒并保留两位小数

    # 计算DES加密和解密的耗时
    des3_enc_time, des3_dec_time = des3_enc_dec(img)
    des3_enc_time, des3_dec_time = (round(des3_enc_time * 1000, 2), round(des3_dec_time * 1000, 2))  # 转换为毫秒并保留两位小数

    # 计算AES加密和解密的耗时
    aes_enc_time, aes_dec_time = aes_enc_dec(img)
    aes_enc_time, aes_dec_time = (round(aes_enc_time * 1000, 2), round(aes_dec_time * 1000, 2))  # 转换为毫秒并保留两位小数

    # 计算不扩展加密和解密的耗时
    no_expand_enc_time, no_expand_dec_time = no_expand_enc_dec(img_gray)
    no_expand_enc_time, no_expand_dec_time = (round(no_expand_enc_time * 1000, 2), round(no_expand_dec_time * 1000, 2))  # 转换为毫秒并保留两位小数

    print(f'Image: {image_file}, size: {ori_size / 1024:.2f} KB')
    print(f'RSA encryption time: {rsa_enc_time} ms, decryption time: {rsa_dec_time} ms')
    print(f'3DES encryption time: {des3_enc_time} ms, decryption time: {des3_dec_time} ms')
    print(f'AES encryption time: {aes_enc_time} ms, decryption time: {aes_dec_time} ms')
    print(f'NE_VC encryption time: {no_expand_enc_time} ms, decryption time: {no_expand_dec_time} ms')

Encrypted image size: 7.26 KB

Image: Airplane.png, size: 40.3408203125 KB

RSA encryption time: 645.11 ms, decryption time: 1576.13 ms

3DES encryption time: 2.02 ms, decryption time: 1.91 ms

AES encryption time: 0.1 ms, decryption time: 0.07 ms

NE_VC encryption time: 1.71 ms, decryption time: 0.0 ms

Encrypted image size: 6.27 KB

Image: Baboon.png, size: 52.6259765625 KB

RSA encryption time: 1020.05 ms, decryption time: 1577.07 ms

3DES encryption time: 2.01 ms, decryption time: 1.82 ms

AES encryption time: 0.1 ms, decryption time: 0.07 ms

NE_VC encryption time: 1.6 ms, decryption time: 0.0 ms

Encrypted image size: 5.94 KB

Image: Barbara.png, size: 43.7177734375 KB

RSA encryption time: 746.45 ms, decryption time: 1568.05 ms

3DES encryption time: 2.01 ms, decryption time: 1.83 ms

AES encryption time: 0.1 ms, decryption time: 0.07 ms

NE_VC encryption time: 1.51 ms, decryption time: 0.0 ms

Encrypted image size: 5.98 KB

Image: Boats.png, size: 40.421875 KB

RSA encryption t